In [ ]:
import time
import copy
import logging
import networkx
from networkx import convert_matrix
from typing import NamedTuple, Optional, Tuple, List, Dict, TypeVar, TypedDict, cast

import numpy
import gym
from gym import spaces
from gym.utils import seeding

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from cyberbattle._env.shared_cyberbattle_env import EnvironmentBounds, AttackerGoal, DefenderGoal, DefenderConstraint
from cyberbattle._env.defender import DefenderAgent,ScanAndReimageCompromisedMachines
from cyberbattle.simulation.model import PortName, PrivilegeLevel
from cyberbattle.simulation import commandcontrol, model, actions
from cyberbattle._env.discriminatedunion import DiscriminatedUnion
from cyberbattle.agents.baseline import agent_wrapper as w
import cyberbattle.agents.baseline.learner as learner
from cyberbattle.agents.baseline.learner import Learner
import cyberbattle.agents.baseline.agent_tabularqlearning as a
import cyberbattle.agents.baseline.agent_ddql as ddqla
import cyberbattle.agents.baseline.agent_dql as dqla
from cyberbattle.defender_agents.trainedDefender import *
import random
random.seed(120394016)
%matplotlib inline
import cyberbattle.agents.baseline.agent_tabularqlearning as a
import cyberbattle.agents.baseline.learner as learner
from cyberbattle.agents.baseline.agent_wrapper import Verbosity
#ql
iteration_count = 10
training_episode_count = 10
eval_episode_count = 20
gamma_sweep = [
    0.015,  # about right
]
import gym

gym_env = gym.make('CyberBattleChain-v0',
    size=12,
    attacker_goal=AttackerGoal(
        own_atleast=0,
        own_atleast_percent=1.0
    ),
    defender_constraint=DefenderConstraint(
        maintain_sla=0.80
    )
)

defender_name = "DefenderName"
defender_agent = DefenderWrapper(gym_env,defender_name)
from cyberbattle.agents.baseline.agent_wrapper import ActionTrackingStateAugmentation, StateAugmentation
from cyberbattle.agents.baseline.plotting import PlotTraining, plot_averaged_cummulative_rewards
import progressbar
import math
from cyberbattle.agents.baseline import learner
import sys
from cyberbattle.simulation.model import FirewallRule
ep = w.EnvironmentBounds.of_identifiers(
    maximum_total_credentials=22,
    maximum_node_count=22,
    identifiers=gym_env.identifiers
)
#cyberbattlechain_defender.render_as_fig()
import cyberbattle.agents.baseline.agent_tabularqlearning as a
import cyberbattle.agents.baseline.learner as learner
from cyberbattle.agents.baseline.agent_wrapper import Verbosity
epsilon = 0.66666666666666
gamma= 0.06925
learning_rate= 0.34444444444444444
epsilon_exponential_decay=3198.830407105263
epsilon_minimum=0.05708811937931034
episode_count=10
iteration_count=10
epsilon_multdecay=0.7777777777777778
statespace = w.HashEncoding(ep, [
            # Feature_discovered_node_count(),
            # Feature_discovered_credential_count(),
            w.Feature_discovered_ports_sliding(ep),
            w.Feature_discovered_nodeproperties_sliding(ep),
            w.Feature_discovered_notowned_node_count(ep, 3)
        ], 5000)  # should not be too small, pick something big to avoid collision

actionspace = w.RavelEncoding(ep, [
            w.Feature_active_node_properties(ep)])
print(statespace)
print(actionspace)
q_matrix = DefenderQMatrix(statespace, actionspace,gym_env)

In [ ]:
def run_training_with_hyperparameters(gym_env, environment_properties, statespace, actionspace, 
                                      epsilon, gamma, learning_rate, epsilon_exponential_decay, epsilon_minimum, episode_count, iteration_count, epsilon_multdecay):
    q_matrix = DefenderQMatrix(statespace, actionspace, gym_env)
    learner = DefenderQLearner(gym_env, defender_agent, environment_properties, q_matrix,
                               epsilon=epsilon, gamma=gamma, learning_rate=learning_rate)
    
    trained_defender = epsilon_greedy_defender_training(
        cyberbattle_gym_env=gym_env,
        environment_properties=environment_properties,
        learner=learner,
        episode_count=episode_count,
        iteration_count=iteration_count,  
        epsilon=epsilon,
        epsilon_exponential_decay=epsilon_exponential_decay,
        epsilon_minimum=epsilon_minimum,
        epsilon_multdecay=epsilon_multdecay,
        verbosity=logging.INFO,
        render=True,
        plot_episodes_length=True,
        title="Defender Q-Learning Training"
    )
    total_reward = sum([sum(rewards) for rewards in trained_defender['all_episodes_rewards']])
    return total_reward

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('seaborn-darkgrid')

# Baseline

# Increased

In [ ]:
import cyberbattle.agents.baseline.agent_tabularqlearning as a
import cyberbattle.agents.baseline.learner as learner
from cyberbattle.agents.baseline.agent_wrapper import Verbosity
epsilon = 0.66666666666666
gamma= 0.06925
learning_rate= 0.34444444444444444
epsilon_exponential_decay=3198.830407105263
epsilon_minimum=0.05708811937931034
episode_count=50
iteration_count=200
epsilon_multdecay=0.7777777777777778
statespace = w.HashEncoding(ep, [
            # Feature_discovered_node_count(),
            # Feature_discovered_credential_count(),
            w.Feature_discovered_ports_sliding(ep),
            w.Feature_discovered_nodeproperties_sliding(ep),
            w.Feature_discovered_notowned_node_count(ep, 3)
        ], 5000) 

actionspace = w.RavelEncoding(ep, [
            w.Feature_active_node_properties(ep)])
print(statespace)
print(actionspace)
q_matrix = DefenderQMatrix(statespace, actionspace,gym_env)
learner2 = DefenderQLearner(gym_env, defender_agent, ep, q_matrix, epsilon=0.9, gamma=0.015, learning_rate=.07)
random.seed(120394016)
#%matplotlib inline

trained_defender = epsilon_greedy_defender_training(
        cyberbattle_gym_env=gym_env,
        environment_properties=ep,
        learner=learner2,
        episode_count=episode_count, 
        iteration_count=iteration_count,
        epsilon=epsilon,
        epsilon_exponential_decay=epsilon_exponential_decay,
        epsilon_minimum=epsilon_minimum,
        epsilon_multdecay=epsilon_multdecay,
        verbosity=logging.INFO,
        render=True,
        plot_episodes_length=True,
        title="Defender Q-Learning Training"
    )

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

class SimulationTracker:
    def __init__(self):
        self.attacker_rewards = []
        self.defender_rewards = []
        self.actions_taken = {'attacker': [], 'defender': []}
        self.exploration_actions = {'attacker': 0, 'defender': 0}
        self.exploitation_actions = {'attacker': 0, 'defender': 0}
        self.episode_lengths = []
        self.reward_variability = []
        self.state_visitation_frequencies = {}
        self.temporal_differences = []
        self.infected_nodes = []

    def log_episode(self, attacker_reward, defender_reward, episode_length, episode_rewards):
        self.attacker_rewards.append(attacker_reward)
        self.defender_rewards.append(defender_reward)
        self.episode_lengths.append(episode_length)
        self.reward_variability.append(np.var(episode_rewards))
        #self.temporal_differences.append(temporal_difference)

    def log_action(self, role, action, exploration):
        action_str = str(action)  
        self.actions_taken[role].append(action_str)
        if exploration:
            self.exploration_actions[role] += 1
        else:
            self.exploitation_actions[role] += 1


    def log_state_visitation(self, state):
        if state not in self.state_visitation_frequencies:
            self.state_visitation_frequencies[state] = 1
        else:
            self.state_visitation_frequencies[state] += 1

    def log_infected_nodes(self, nodes_count):
        self.infected_nodes.append(nodes_count)

    def print_summary(self):
        # Summary statistics
        print("Simulation Summary:")
        print(f"Average Attacker Reward: {np.mean(self.attacker_rewards)}")
        print(f"Average Defender Reward: {np.mean(self.defender_rewards)}")
        print(f"Average Episode Length: {np.mean(self.episode_lengths)} steps")
        print(f"Average Reward Variability: {np.mean(self.reward_variability)}")
        print(f"Average Temporal Difference: {np.mean(self.temporal_differences)}")
        print(f"Average Infected Nodes per Episode: {np.mean(self.infected_nodes)}")

        #exploration vs. exploitation
        print("\nExploration vs. Exploitation:")
        for role in ['attacker', 'defender']:
            total_actions = self.exploration_actions[role] + self.exploitation_actions[role]
            if total_actions > 0:
                print(f"{role.capitalize()} Exploration: {self.exploration_actions[role] / total_actions:.2f}")
                print(f"{role.capitalize()} Exploitation: {self.exploitation_actions[role] / total_actions:.2f}")

        #action Distributions
        self._print_action_distributions()

        self._plot_rewards()

    def _print_action_distributions(self):
        print("\nAction Distributions:")
        for role, actions in self.actions_taken.items():
            # actions are already converted to strings by log_action
            unique_actions, counts = np.unique(actions, return_counts=True)
            print(f"{role.capitalize()} Actions:")
            for action, count in zip(unique_actions, counts):
                print(f"Action {action}: {count} times")


    def _plot_rewards(self):
        plt.figure(figsize=(10, 5))
        plt.plot(self.attacker_rewards, label='Attacker Rewards')
        plt.plot(self.defender_rewards, label='Defender Rewards')
        plt.xlabel('Episode')
        plt.ylabel('Reward')
        plt.title('Rewards per Episode for Attacker and Defender')
        plt.legend()
        plt.show()


In [ ]:
import time
import copy
import logging
import networkx
from networkx import convert_matrix
from typing import NamedTuple, Optional, Tuple, List, Dict, TypeVar, TypedDict, cast

import numpy
import gym
from gym import spaces
from gym.utils import seeding

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from cyberbattle._env.shared_cyberbattle_env import EnvironmentBounds, AttackerGoal, DefenderGoal, DefenderConstraint
from cyberbattle._env.defender import DefenderAgent,ScanAndReimageCompromisedMachines
from cyberbattle.simulation.model import PortName, PrivilegeLevel
from cyberbattle.simulation import commandcontrol, model, actions
from cyberbattle._env.discriminatedunion import DiscriminatedUnion
from cyberbattle.agents.baseline import agent_wrapper as w
import cyberbattle.agents.baseline.learner as learner
from cyberbattle.agents.baseline.learner import Learner
import cyberbattle.agents.baseline.agent_tabularqlearning as a
import cyberbattle.agents.baseline.agent_ddql as ddqla
import cyberbattle.agents.baseline.agent_dql as dqla
from cyberbattle.defender_agents.trainedDefender import *
import random
random.seed(120394016)
%matplotlib inline
import cyberbattle.agents.baseline.agent_tabularqlearning as a
import cyberbattle.agents.baseline.learner as learner
from cyberbattle.agents.baseline.agent_wrapper import Verbosity
#ql
iteration_count = 10
training_episode_count = 10
eval_episode_count = 20
gamma_sweep = [
    0.015,  # about right
]
import gym

gym_env = gym.make('CyberBattleChain-v0',
    size=12,
    attacker_goal=AttackerGoal(
        own_atleast=0,
        own_atleast_percent=1.0
    ),
    defender_constraint=DefenderConstraint(
        maintain_sla=0.80
    )
)

defender_name = "DefenderName"
defender_agent = DefenderWrapper(gym_env,defender_name)
from cyberbattle.agents.baseline.agent_wrapper import ActionTrackingStateAugmentation, StateAugmentation
from cyberbattle.agents.baseline.plotting import PlotTraining, plot_averaged_cummulative_rewards
import progressbar
import math
from cyberbattle.agents.baseline import learner
import sys
from cyberbattle.simulation.model import FirewallRule
ep = w.EnvironmentBounds.of_identifiers(
    maximum_total_credentials=22,
    maximum_node_count=22,
    identifiers=gym_env.identifiers
)
#cyberbattlechain_defender.render_as_fig()
import cyberbattle.agents.baseline.agent_tabularqlearning as a
import cyberbattle.agents.baseline.learner as learner
from cyberbattle.agents.baseline.agent_wrapper import Verbosity
epsilon = 0.66666666666666
gamma= 0.06925
learning_rate= 0.34444444444444444
epsilon_exponential_decay=3198.830407105263
epsilon_minimum=0.05708811937931034
episode_count=10
iteration_count=10
epsilon_multdecay=0.7777777777777778
statespace = w.HashEncoding(ep, [
            # Feature_discovered_node_count(),
            # Feature_discovered_credential_count(),
            w.Feature_discovered_ports_sliding(ep),
            w.Feature_discovered_nodeproperties_sliding(ep),
            w.Feature_discovered_notowned_node_count(ep, 3)
        ], 5000)  # should not be too small, pick something big to avoid collision

actionspace = w.RavelEncoding(ep, [
            w.Feature_active_node_properties(ep)])
print(statespace)
print(actionspace)
q_matrix = DefenderQMatrix(statespace, actionspace,gym_env)

In [ ]:
random.seed(120394016)
# Parameter ranges
epsilon = 0.9629629629629622
#gamma=np.linspace(0.05,0.09,10)
gamma= 0.05888888888888889
#learning_rate=np.linspace(0.3,.35,10)
learning_rate=0.3222222222222222
#epsilon_decay=np.linspace(2333.33333,5000,20)
epsilon_exponential_decay=5000
#epsilon_minimum=np.linspace(0.01,.07777777,10)
epsilon_minimum=0.0513409942068965
episode_count=50
iteration_count=200
#epsilon_multdecay=np.linspace(0.3,0.875,20)
epsilon_multdecay=0.6631578947368421

trained_attacker = learner.epsilon_greedy_search(
        gym_env,
        ep,
        a.QTabularLearner(ep, gamma=0.05888888888888889, learning_rate=0.3222222222222222, exploit_percentile=100),
        episode_count=episode_count,
        iteration_count=iteration_count,
        epsilon=epsilon,
        render=True,
        epsilon_multdecay=epsilon_multdecay,  # 0.999,
        epsilon_exponential_decay=epsilon_exponential_decay,
        epsilon_minimum=epsilon_minimum,
        verbosity=Verbosity.Quiet,
        title="Q-learning"
    )

In [ ]:
import gym

gym_env = gym.make('CyberBattleChain-v0',
    size=12,
    attacker_goal=AttackerGoal(
        own_atleast=0,
        own_atleast_percent=1.0
    ),
    defender_constraint=DefenderConstraint(
        maintain_sla=0.80
    )
)

defender_name = "DefenderName" 
defender_agent = DefenderWrapper(gym_env,defender_name)

gym_environment = gym.make('CyberBattleChain-v0',
    size=12,
    attacker_goal=AttackerGoal(
        own_atleast=0,
        own_atleast_percent=1.0
    ),
    defender_constraint=DefenderConstraint(
        maintain_sla=0.80
    ),
    defender_agent=defender_agent
)

#this was made due to issues where the attacker q learner needed to use a wrapper environment
from cyberbattle.agents.baseline.agent_wrapper import AgentWrapper, StateAugmentation
def wrap_environment(env):
    initial_observation = env.reset()  
    state = StateAugmentation(initial_observation)  
    return AgentWrapper(env, state)



In [ ]:
attacker_learner = trained_attacker['learner']
defender_learner = trained_defender['learner']

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

number_of_episodes = 50
max_steps_per_episode = 200
attacker_epsilon = 0.1
defender_epsilon = 0.1

attacker_rewards_per_episode = []
defender_rewards_per_episode = []
win_rates = {'attacker': [], 'defender': []}
actions_taken_count = {'attacker': [], 'defender': []}
exploration_vs_exploitation = {'exploration': [], 'exploitation': []}
episode_lengths = []
state_visitation_frequencies = {}  
reward_variability = []
defender_successful_actions = []
defender_failed_actions = []
defender_actions_taken = []
tracker = SimulationTracker() #instantiate simulation tracker
wrapped_env = wrap_environment(gym_environment)
win_counts = {'attacker': 0, 'defender': 0}

random.seed(120394016)
for episode in range(number_of_episodes):
    observation = wrapped_env.reset()
    attacker_total_reward = 0
    defender_total_reward = 0
    steps_this_episode=0
    episode_rewards = [] 
    for step in range(max_steps_per_episode):
        steps_this_episode +=1
        # Attacker's Turn
        exploration = np.random.random() < attacker_epsilon
        if exploration:
            _, attacker_action, _ = attacker_learner.explore(wrapped_env)
        else:
            _, attacker_action, _ = attacker_learner.exploit(wrapped_env, observation)
        #print(attacker_action)
        if attacker_action is not None:
            observation, attacker_reward, done, _ = gym_environment.step(attacker_action)
            attacker_total_reward += attacker_reward
            print(_)
            actions_taken_count['attacker'].append(attacker_action)
            episode_rewards.append(attacker_reward)  
            tracker.log_action('attacker', attacker_action, exploration)
            #actions_taken_count['attacker'][attacker_action] = actions_taken_count['attacker'].get(attacker_action, 0) + 1


        # Defender's Turn
        exploration = np.random.random() < defender_epsilon
        if exploration:
            _, defender_action, _ = defender_learner.explore(wrapped_env)
        else:
            _, defender_action, _ = defender_learner.exploit(wrapped_env, observation)

        if defender_action is not None:
            observation, defender_reward, done, _ = gym_environment.step(defender_action)
            #action= _['defender_info']['action']
            defender_total_reward += defender_reward
            actions_taken_count['defender'].append(defender_action)  
            episode_rewards.append(defender_reward)  
            tracker.log_action('defender', defender_action, exploration)
            #actions_taken_count['defender'][defender_action] = actions_taken_count['defender'].get(defender_action, 0) + 1
        #logging
        #after each episode
        tracker.log_episode(attacker_total_reward, defender_total_reward, steps_this_episode, episode_rewards)
        tracker.print_summary()
        tracker._plot_rewards()

        if done:
            break
            
    defender_info = _['defender_info']  
    if defender_info:  
        defender_actions_taken.append(defender_info['performance_metrics']['actions_taken'])
        defender_successful_actions.append(defender_info['performance_metrics']['successful_actions'])
        defender_failed_actions.append(defender_info['performance_metrics']['failed_actions'])

    attacker_rewards_per_episode.append(attacker_total_reward)
    defender_rewards_per_episode.append(defender_total_reward)
    episode_lengths.append(steps_this_episode)
    reward_variability.append(np.var(episode_rewards))
    print(f"Episode {episode + 1}: Attacker Reward = {attacker_total_reward}, Defender Reward = {defender_total_reward}")

wrapped_env.close()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

def moving_average(data, window_size):
    return np.convolve(data, np.ones(window_size)/window_size, mode='valid')

number_of_episodes = len(attacker_rewards_per_episode) 

window_size = 3
attacker_ma = moving_average(attacker_rewards_per_episode, window_size)
defender_ma = moving_average(defender_rewards_per_episode, window_size)

# Calculate the mean and 95% CI for both attacker and defender rewards
attacker_mean = np.mean(attacker_rewards_per_episode)
defender_mean = np.mean(defender_rewards_per_episode)
attacker_ci = 1.96 * stats.sem(attacker_rewards_per_episode)
defender_ci = 1.96 * stats.sem(defender_rewards_per_episode)

plt.figure(figsize=(14, 7))

#original rewards
plt.plot(attacker_rewards_per_episode, 'o-', color='skyblue', alpha=0.5, label='Attacker Rewards')
plt.plot(defender_rewards_per_episode, 'x-', color='lightcoral', alpha=0.5, label='Defender Rewards')

#moving averages
plt.plot(range(window_size-1, len(attacker_ma)+window_size-1), attacker_ma, color='navy', label='Attacker MA')
plt.plot(range(window_size-1, len(defender_ma)+window_size-1), defender_ma, color='darkred', label='Defender MA')

#mean lines
plt.axhline(attacker_mean, color='blue', linestyle='--', label=f'Attacker Mean: {attacker_mean:.2f}')
plt.axhline(defender_mean, color='red', linestyle='--', label=f'Defender Mean: {defender_mean:.2f}')

#Confidence interval bands around the means
#plt.fill_betweenx([0, max(max(attacker_rewards_per_episode), max(defender_rewards_per_episode))], attacker_mean - attacker_ci, attacker_mean + attacker_ci, color='blue', alpha=0.1)
#plt.fill_betweenx([0, max(max(attacker_rewards_per_episode), max(defender_rewards_per_episode))], defender_mean - defender_ci, defender_mean + defender_ci, color='red', alpha=0.1)

plt.title('Attacker and Defender Rewards per Episode', fontsize=16)
plt.xlabel('Episode', fontsize=14)
plt.ylabel('Rewards', fontsize=14)
plt.legend(loc='upper right', fontsize=12)
plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.7)
plt.tight_layout()

plt.show()


In [ ]:
#calculate means
attacker_mean = np.mean(attacker_rewards_per_episode)
defender_mean = np.mean(defender_rewards_per_episode)

plt.figure(figsize=(12, 6))
plt.plot(attacker_rewards_per_episode, label='Attacker Rewards', color='blue', marker='o')
plt.plot(defender_rewards_per_episode, label='Defender Rewards', color='red', marker='x')

plt.axhline(y=attacker_mean, color='skyblue', linestyle='--', label=f'Attacker Mean: {attacker_mean:.2f}')
plt.axhline(y=defender_mean, color='darkred', linestyle='--', label=f'Defender Mean: {defender_mean:.2f}')

plt.title('Rewards Per Episode for Attacker and Defender')
plt.xlabel('Episode')
plt.ylabel('Total Reward')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
win_rates['attacker'] = np.cumsum(np.array(attacker_rewards_per_episode) > np.array(defender_rewards_per_episode)) / np.arange(1, number_of_episodes + 1)
win_rates['defender'] = 1 - win_rates['attacker']

attacker_win_rate=win_rates['attacker'] 
defender_win_rate=win_rates['defender']

#calculate means
attacker_win_rate_mean = np.mean(attacker_win_rate)
defender_win_rate_mean = np.mean(defender_win_rate)

plt.figure(figsize=(12, 6))
plt.plot(attacker_win_rate, label='Attacker Win Rate', color='blue')
plt.plot(defender_win_rate, label='Defender Win Rate', color='red')

plt.axhline(y=attacker_win_rate_mean, color='blue', linestyle='--', label=f'Attacker Mean: {attacker_win_rate_mean:.2f}')
plt.axhline(y=defender_win_rate_mean, color='red', linestyle='--', label=f'Defender Mean: {defender_win_rate_mean:.2f}')

plt.title('Win Rates Over Episodes')
plt.xlabel('Episode')
plt.ylabel('Win Rate')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 6))
plt.plot(win_rates['attacker'], label='Attacker Win Rate', color='blue')
plt.plot(win_rates['defender'], label='Defender Win Rate', color='red')
plt.title('Win Rates Over Episodes')
plt.xlabel('Episode')
plt.ylabel('Win Rate')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()




In [ ]:
defender_win_rate

#Episode Lengths Over Time:

In [ ]:
plt.figure(figsize=(14, 7))
plt.plot(episode_lengths, label='Episode Lengths')
plt.title('Episode Lengths Over Time')
plt.xlabel('Episode')
plt.ylabel('Length (Number of Steps)')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats

def moving_average(data, window_size):
    """Calculate the moving average using a sliding window approach."""
    return np.convolve(data, np.ones(window_size)/window_size, mode='valid')

def calculate_confidence_interval(data, confidence=0.95):
    """Calculate the confidence interval for the given data."""
    n = len(data)
    mean = np.mean(data)
    sem = stats.sem(data)  #standard error of the mean
    h = sem * stats.t.ppf((1 + confidence) / 2., n-1)  #margin of error
    return mean - h, mean + h

window_size = 5  
ma_successful = moving_average(defender_successful_actions, window_size)
ma_failed = moving_average(defender_failed_actions, window_size)

ci_lower_successful, ci_upper_successful = calculate_confidence_interval(ma_successful)
ci_lower_failed, ci_upper_failed = calculate_confidence_interval(ma_failed)

plt.figure(figsize=(14, 7))
plt.style.use('seaborn-darkgrid')

plt.fill_between(range(window_size-1, len(ma_successful)+window_size-1), ci_lower_successful, ci_upper_successful, color='blue', alpha=0.1, label='Confidence Interval (Successful)')
plt.fill_between(range(window_size-1, len(ma_failed)+window_size-1), ci_lower_failed, ci_upper_failed, color='red', alpha=0.1, label='Confidence Interval (Failed)')
plt.plot(range(window_size-1, len(ma_successful)+window_size-1), ma_successful, label='Moving Average (Successful)', color='blue', linewidth=2)
plt.plot(range(window_size-1, len(ma_failed)+window_size-1), ma_failed, label='Moving Average (Failed)', color='red', linewidth=2)

mean_successful = np.mean(defender_successful_actions)
mean_failed = np.mean(defender_failed_actions)
plt.axhline(y=mean_successful, color='blue', linestyle='--', linewidth=2, label=f'Mean Successful Actions: {mean_successful:.2f}')
plt.axhline(y=mean_failed, color='red', linestyle='--', linewidth=2, label=f'Mean Failed Actions: {mean_failed:.2f}')

plt.title('Defender Action Success vs. Failure Rates with Moving Averages', fontsize=16)
plt.xlabel('Episode', fontsize=14)
plt.ylabel('Number of Actions', fontsize=14)
plt.legend(loc='upper right', fontsize=12, bbox_to_anchor=(1.25, 1))
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid(True, which='major', linestyle='--', linewidth=0.5, alpha=0.7)
plt.minorticks_on()
plt.grid(True, which='minor', linestyle=':', linewidth=0.5, alpha=0.5)

plt.tight_layout(rect=[0, 0, 0.75, 1]) 
plt.show()


In [ ]:
print(defender_successful_actions)
print(defender_actions_taken)

In [ ]:
defender_win_rate

In [ ]:
#comparing defender win rates
baseline_reward_structure=[1.        , 0.5       , 0.33333333, 0.25      , 0.4       ,
       0.5       , 0.57142857, 0.5       , 0.55555556, 0.6       ,
       0.63636364, 0.66666667, 0.69230769, 0.64285714, 0.66666667,
       0.6875    , 0.64705882, 0.61111111, 0.63157895, 0.6       ,
       0.57142857, 0.54545455, 0.52173913, 0.5       , 0.52      ,
       0.53846154, 0.55555556, 0.57142857, 0.55172414, 0.53333333,
       0.51612903, 0.53125   , 0.54545455, 0.55882353, 0.54285714,
       0.52777778, 0.51351351, 0.5       , 0.48717949, 0.5       ,
       0.51219512, 0.5       , 0.48837209, 0.47727273, 0.48888889,
       0.47826087, 0.4893617 , 0.5       , 0.51020408, 0.52      ]

modified_reward_structure=[1.        , 1.        , 0.66666667, 0.5       , 0.6       ,
       0.66666667, 0.71428571, 0.625     , 0.66666667, 0.7       ,
       0.72727273, 0.75      , 0.76923077, 0.71428571, 0.73333333,
       0.75      , 0.70588235, 0.66666667, 0.68421053, 0.65      ,
       0.61904762, 0.63636364, 0.60869565, 0.58333333, 0.6       ,
       0.61538462, 0.62962963, 0.64285714, 0.62068966, 0.6       ,
       0.58064516, 0.59375   , 0.60606061, 0.61764706, 0.6       ,
       0.58333333, 0.56756757, 0.55263158, 0.53846154, 0.55      ,
       0.56097561, 0.54761905, 0.53488372, 0.52272727, 0.53333333,
       0.52173913, 0.53191489, 0.54166667, 0.55102041, 0.56      ]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

defender_win_rate_new = np.array([1., 1., 0.66666667, 0.5, 0.6, 0.66666667, 0.71428571, 0.625, 0.66666667, 0.7, 0.72727273, 0.75, 0.76923077, 0.71428571, 0.73333333, 0.75, 0.70588235, 0.66666667, 0.68421053, 0.65, 0.61904762, 0.63636364, 0.60869565, 0.58333333, 0.6, 0.61538462, 0.62962963, 0.64285714, 0.62068966, 0.6, 0.58064516, 0.59375, 0.60606061, 0.61764706, 0.6, 0.58333333, 0.56756757, 0.55263158, 0.53846154, 0.55, 0.56097561, 0.54761905, 0.53488372, 0.52272727, 0.53333333, 0.52173913, 0.53191489, 0.54166667, 0.55102041, 0.56])
defender_win_rate_old = np.array([1., 0.5, 0.33333333, 0.25, 0.4, 0.5, 0.57142857, 0.5, 0.55555556, 0.6, 0.63636364, 0.66666667, 0.69230769, 0.64285714, 0.66666667, 0.6875, 0.64705882, 0.61111111, 0.63157895, 0.6, 0.57142857, 0.54545455, 0.52173913, 0.5, 0.52, 0.53846154, 0.55555556, 0.57142857, 0.55172414, 0.53333333, 0.51612903, 0.53125, 0.54545455, 0.55882353, 0.54285714, 0.52777778, 0.51351351, 0.5, 0.48717949, 0.5, 0.51219512, 0.5, 0.48837209, 0.47727273, 0.48888889, 0.47826087, 0.4893617, 0.5, 0.51020408, 0.52])

episodes = np.arange(1, 51)

#calculate statistical significance
t_stat, p_value = stats.ttest_ind(defender_win_rate_new, defender_win_rate_old)

plt.figure(figsize=(12, 7))
plt.plot(episodes, defender_win_rate_new, label='New Reward Structure', color='green', marker='o')
plt.plot(episodes, defender_win_rate_old, label='Old Reward Structure', color='red', marker='x')
plt.title('Defender Win Rate: New vs. Old Reward Structure', fontsize=16)
plt.xlabel('Episode', fontsize=14)
plt.ylabel('Win Rate', fontsize=14)
plt.legend(loc='best', fontsize=12)
plt.grid(True, which='both', linestyle='--', alpha=0.7)

#highlighting statistical significance
plt.figtext(0.5, 0.01, f'p-value = {p_value:.2e}', ha='center', fontsize=12, bbox={"facecolor":"orange", "alpha":0.5, "pad":5})

plt.tight_layout()
plt.show()


In [ ]:
#calculate the means
mean_new = np.mean(defender_win_rate_new)
mean_old = np.mean(defender_win_rate_old)

#calculate statistical significance
t_stat, p_value = stats.ttest_ind(defender_win_rate_new, defender_win_rate_old)

plt.figure(figsize=(12, 7))
plt.plot(episodes, defender_win_rate_new, label='New Reward Structure', color='green', marker='o')
plt.plot(episodes, defender_win_rate_old, label='Old Reward Structure', color='red', marker='x')

plt.axhline(y=mean_new, color='darkgreen', linestyle='--', linewidth=1.5, label=f'Mean New (μ={mean_new:.2f})')
plt.axhline(y=mean_old, color='darkred', linestyle='--', linewidth=1.5, label=f'Mean Old (μ={mean_old:.2f})')

plt.title('Defender Win Rate: New vs. Old Reward Structure', fontsize=16)
plt.xlabel('Episode', fontsize=14)
plt.ylabel('Win Rate', fontsize=14)
plt.legend(loc='best', fontsize=12)
plt.grid(True, which='both', linestyle='--', alpha=0.7)

plt.text(x=0.5, y=0.01, s=f'p-value: {p_value:.2e}', horizontalalignment='center', fontsize=12)

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

defender_win_rate_new = np.array([1., 1., 0.66666667, 0.5, 0.6, 0.66666667, 0.71428571, 0.625, 0.66666667, 0.7, 0.72727273, 0.75, 0.76923077, 0.71428571, 0.73333333, 0.75, 0.70588235, 0.66666667, 0.68421053, 0.65, 0.61904762, 0.63636364, 0.60869565, 0.58333333, 0.6, 0.61538462, 0.62962963, 0.64285714, 0.62068966, 0.6, 0.58064516, 0.59375, 0.60606061, 0.61764706, 0.6, 0.58333333, 0.56756757, 0.55263158, 0.53846154, 0.55, 0.56097561, 0.54761905, 0.53488372, 0.52272727, 0.53333333, 0.52173913, 0.53191489, 0.54166667, 0.55102041, 0.56])
defender_win_rate_old = np.array([1., 0.5, 0.33333333, 0.25, 0.4, 0.5, 0.57142857, 0.5, 0.55555556, 0.6, 0.63636364, 0.66666667, 0.69230769, 0.64285714, 0.66666667, 0.6875, 0.64705882, 0.61111111, 0.63157895, 0.6, 0.57142857, 0.54545455, 0.52173913, 0.5, 0.52, 0.53846154, 0.55555556, 0.57142857, 0.55172414, 0.53333333, 0.51612903, 0.53125, 0.54545455, 0.55882353, 0.54285714, 0.52777778, 0.51351351, 0.5, 0.48717949, 0.5, 0.51219512, 0.5, 0.48837209, 0.47727273, 0.48888889, 0.47826087, 0.4893617, 0.5, 0.51020408, 0.52])

episodes = np.arange(1, 51)

mean_new = np.mean(defender_win_rate_new)
mean_old = np.mean(defender_win_rate_old)

#calculate statistical significance
t_stat, p_value = stats.ttest_ind(defender_win_rate_new, defender_win_rate_old)

plt.figure(figsize=(12, 7))
plt.plot(episodes, defender_win_rate_new, label='New Reward Structure', color='green', marker='o')
plt.plot(episodes, defender_win_rate_old, label='Old Reward Structure', color='red', marker='x')

#mean lines
plt.axhline(y=mean_new, color='darkgreen', linestyle='--', linewidth=1.5, label=f'New Mean: {mean_new:.2f}')
plt.axhline(y=mean_old, color='darkred', linestyle='--', linewidth=1.5, label=f'Old Mean: {mean_old:.2f}')

plt.title('Defender Win Rate: New vs. Old Reward Structure', fontsize=16)
plt.xlabel('Episode', fontsize=14)
plt.ylabel('Win Rate', fontsize=14)
plt.legend(loc='best', fontsize=12)
plt.grid(True, which='both', linestyle='--', alpha=0.7)

plt.figtext(0.5, 0.01, f'p-value = {p_value:.2e}', ha='center', fontsize=12, bbox={"facecolor":"orange", "alpha":0.5, "pad":5})

plt.tight_layout()
plt.show()

This visualization compares the defender's win rate over 50 episodes between a new reward structure and an old one. The green line represents the win rate under the new reward structure, while the red line shows the win rate with the old structure. Each point on the graph corresponds to a specific episode, showing how the win rate evolves over time for both structures.

The statistical significance of the difference between these two win rates is indicated by the p-value, which is displayed at the bottom of the plot. A p-value measures the probability of observing the data—or something more extreme—if the null hypothesis is true. In this context, the null hypothesis states that there's no difference in the win rates provided by the new and old reward structures.

A small p-value (typically ≤ 0.05) would suggest that there's a statistically significant difference between the win rates under the two reward structures, implying that the changes in the reward structure have a meaningful impact on the defender's performance. Conversely, a large p-value suggests that any difference in win rates could be due to random chance, and the new reward structure might not significantly improve the defender's performance.

Given that this visualization includes a p-value, the p-value is small, it indicates that the new reward structure significantly impacts the defender's win rate compared to the old structure.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Provided win rate arrays
defender_win_rate_new = np.array([1., 1., 0.66666667, 0.5, 0.6, 0.66666667, 0.71428571, 0.625, 0.66666667, 0.7, 0.72727273, 0.75, 0.76923077, 0.71428571, 0.73333333, 0.75, 0.70588235, 0.66666667, 0.68421053, 0.65, 0.61904762, 0.63636364, 0.60869565, 0.58333333, 0.6, 0.61538462, 0.62962963, 0.64285714, 0.62068966, 0.6, 0.58064516, 0.59375, 0.60606061, 0.61764706, 0.6, 0.58333333, 0.56756757, 0.55263158, 0.53846154, 0.55, 0.56097561, 0.54761905, 0.53488372, 0.52272727, 0.53333333, 0.52173913, 0.53191489, 0.54166667, 0.55102041, 0.56])
defender_win_rate_old = np.array([1., 0.5, 0.33333333, 0.25, 0.4, 0.5, 0.57142857, 0.5, 0.55555556, 0.6, 0.63636364, 0.66666667, 0.69230769, 0.64285714, 0.66666667, 0.6875, 0.64705882, 0.61111111, 0.63157895, 0.6, 0.57142857, 0.54545455, 0.52173913, 0.5, 0.52, 0.53846154, 0.55555556, 0.57142857, 0.55172414, 0.53333333, 0.51612903, 0.53125, 0.54545455, 0.55882353, 0.54285714, 0.52777778, 0.51351351, 0.5, 0.48717949, 0.5, 0.51219512, 0.5, 0.48837209, 0.47727273, 0.48888889, 0.47826087, 0.4893617, 0.5, 0.51020408, 0.52])

# Plotting
plt.figure(figsize=(14, 7))
episodes = np.arange(1, 51)  
plt.plot(episodes, defender_win_rate_new, label='New Reward Structure', color='green', marker='o')
plt.plot(episodes, defender_win_rate_old, label='Old Reward Structure', color='red', marker='x')

# Mean lines
mean_new = np.mean(defender_win_rate_new)
mean_old = np.mean(defender_win_rate_old)
plt.axhline(y=mean_new, color='green', linestyle='--', linewidth=2, label=f'Mean New: {mean_new:.2f}')
plt.axhline(y=mean_old, color='red', linestyle='--', linewidth=2, label=f'Mean Old: {mean_old:.2f}')

# Annotations for mean values
plt.annotate(f'Mean: {mean_new:.2f}', xy=(50, mean_new), xytext=(51, mean_new),
             arrowprops=dict(facecolor='green', shrink=0.05), color='green')
#plt.annotate(f'Mean: {mean_old:.2f}', xy)
